In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import numpy as np
import cv2
import os
import imageio.v2 as imageio  # v2 for imwrite consistency


In [25]:
def video_to_array(video_path: str) -> np.ndarray:
    """
    Read all frames from a video into a NumPy array.

    Args:
        video_path: Path to input video file.

    Returns:
        np.ndarray of shape (num_frames, H, W, 3), dtype=uint8 (BGR order).
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {video_path}")

    frames = []
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frames.append(frame)

    cap.release()
    return np.stack(frames, axis=0)

# output_path = "/data2/saikiran.tedla/hdrvideo/diff/stuttgart_output.mp4"
# video_array = video_to_array(output_path)/255


np_path = "/data2/saikiran.tedla/hdrvideo/diff/stuttgart_output.npy"
video_array = np.load(np_path)  # Load from .npy file if available
video_array = video_array.transpose (1,2,3,0) * 0.5 + 0.5  # Convert from (C,T,H,W) to (T,H,W,C)
video_array = video_array[:,:,:,::-1]  # Convert from RGB to BGR
print("Loaded video shape:", video_array.shape)
#print max and min of video_array
print("Max value in video_array:", np.max(video_array))
print("Min value in video_array:", np.min(video_array))

# #np.mean(np.abs(video_array - quantized_video_array))  # Check difference between two arrays


Loaded video shape: (13, 480, 832, 3)
Max value in video_array: 1.0
Min value in video_array: 0.0


In [26]:
print(f"Video shape: {video_array.shape}, dtype: {video_array.dtype}")

from utils import process_bracketed_video



def output_hdr_video(hdr_video, out_folder):
    os.makedirs(out_folder, exist_ok=True)
    #write out each frame of hdr_video to path as .rad file
    N, H, W, _ = hdr_video.shape
    for i in range(N):
        frame = hdr_video[i]
        filename = f"{out_folder}/frame_{i:04d}.hdr"
        print("Mean of frame:", np.mean(frame))
        cv2.imwrite(filename, frame)  # Save as .hdr image
        
hdr_video = process_bracketed_video(video_array)
output_hdr_video(hdr_video, "/data2/saikiran.tedla/hdrvideo/diff/my_hdr_frames")
    




Video shape: (13, 480, 832, 3), dtype: float32
Mean of frame: 1.1839157
Mean of frame: 1.169149
Mean of frame: 1.3717998
Mean of frame: 1.1782614
